# Seq2Seq Q&A Chatbot 구현

### 데이터 취득

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/refs/heads/master/ChatbotData.csv')
df = df[['Q', 'A']]
df

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.
...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
11820,흑기사 해주는 짝남.,설렜겠어요.
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
dtypes: object(2)
memory usage: 184.9+ KB


### 데이터 전처리

##### 토커나이저 학습 (sentencepiece 활용)

- 접두사, 접미사 처리 (bos, eos)
     - Train() 인자 cmd 옵션을 추가 (`<bos>, <eos>, <pad>, <oov>...`)<br>
       set_encode_extra_options(':')<br>
       set_encode_extra_options('bos:')<br>
       set_encode_extra_options(':eos')<br>
       set_encode_extra_options('bos:eos')

In [3]:
df[['Q','A']].stack().to_csv("train.txt", index=False, header=False)

In [4]:
import sentencepiece as spt

spt.SentencePieceTrainer.Train(
    input='train.txt',
    model_prefix='qna',
    vocab_size=5000
)

In [ ]:
import sentencepiece as spt

sp = spt.SentencePieceProcessor()
sp.Load("qna.model")

for doc in df['Q'].values[:3]:
    print("원문:", doc)
    print("pieces:", sp.encode_as_pieces(doc)) 
    print("ids:", sp.encode_as_ids(doc))        
    print()

원문: 12시 땡!
pieces: ['▁12', '시', '▁', '땡', '!']
ids: [4230, 298, 4, 4827, 71]

원문: 1지망 학교 떨어졌어
pieces: ['▁1', '지', '망', '▁학교', '▁떨어졌어']
ids: [290, 13, 896, 672, 1740]

원문: 3박4일 놀러가고 싶다
pieces: ['▁3', '박', '4', '일', '▁놀러가고', '▁싶다']
ids: [276, 1285, 2304, 75, 2963, 93]



##### 학습용 데이터 Q_input, A_input, A_target 생성

In [6]:
VOCAB_SIZE = 10000

In [7]:
Q_inputs = []
A_inputs = []
A_tagets = []

for q, a in zip(df['Q'], df['A']):
        a_input = '<bos> ' + a
        a_taget = a + ' <eos>'

        Q_inputs.append(q)
        A_inputs.append(a_input)
        A_tagets.append(a_taget)

len(Q_inputs), len(A_inputs), len(A_tagets)

(11823, 11823, 11823)

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

Q_tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<OOV>')
Q_tokenizer.fit_on_texts(Q_inputs)
Q_inputs_seq = Q_tokenizer.texts_to_sequences(Q_inputs)

Q_inputs_seq[2500:2505]

[[1559, 6061, 6], [1559, 104], [1559, 6062, 6063], [1559, 526], [3248, 3249]]

In [9]:
q_num_words = min(VOCAB_SIZE, len(Q_tokenizer.word_index))
q_max_len = max([len(seq) for seq in Q_inputs_seq])

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [11]:
A_tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<OOV', filters='')
A_tokenizer.fit_on_texts(A_inputs + A_tagets)

A_inputs_seq = A_tokenizer.texts_to_sequences(A_inputs)
A_targets_seq = A_tokenizer.texts_to_sequences(A_tagets)

In [12]:
A_tokenizer.index_word

{1: '<OOV',
 2: '<bos>',
 3: '<eos>',
 4: '거예요.',
 5: '수',
 6: '더',
 7: '같아요.',
 8: '좋은',
 9: '있어요.',
 10: '잘',
 11: '있을',
 12: '많이',
 13: '것',
 14: '게',
 15: '마세요.',
 16: '해보세요.',
 17: '건',
 18: '수도',
 19: '거',
 20: '마음이',
 21: '좋을',
 22: '없어요.',
 23: '시간이',
 24: '그',
 25: '안',
 26: '될',
 27: '것도',
 28: '다른',
 29: '바랄게요.',
 30: '사람',
 31: '돼요.',
 32: '좀',
 33: '저도',
 34: '있는',
 35: '그런',
 36: '보세요.',
 37: '너무',
 38: '하세요.',
 39: '좋아요.',
 40: '않아요.',
 41: '제가',
 42: '바랍니다.',
 43: '그럴',
 44: '할',
 45: '걸',
 46: '드세요.',
 47: '같이',
 48: '해요.',
 49: '좋죠.',
 50: '생각해요.',
 51: '말해보세요.',
 52: '사람이',
 53: '좋겠어요.',
 54: '마음을',
 55: '먼저',
 56: '몰라요.',
 57: '이제',
 58: '하고',
 59: '일이',
 60: '자신을',
 61: '다',
 62: '생각해보세요.',
 63: '하는',
 64: '아니에요.',
 65: '정말',
 66: '좋겠네요.',
 67: '힘든',
 68: '다시',
 69: '마음',
 70: '있으면',
 71: '때',
 72: '한',
 73: '바라요.',
 74: '그게',
 75: '많은',
 76: '주세요.',
 77: '사랑은',
 78: '찾아보세요.',
 79: '말고',
 80: '만큼',
 81: '어떨까요.',
 82: '가보세요.',
 83: '저는',
 84: '아직',
 85: '직접',
 86: '

In [13]:
a_num_words = min(VOCAB_SIZE, len(A_tokenizer.index_word))
a_max_len = min(15, max(len(seq) for seq in A_inputs_seq))

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

Q_inputs_padded = pad_sequences(Q_inputs_seq, maxlen=q_max_len, padding='pre')
A_inputs_padded = pad_sequences(A_inputs_seq, maxlen=a_max_len, padding='post')
A_targets_padded = pad_sequences(A_targets_seq, maxlen=a_max_len, padding='post')

In [15]:
Q_inputs_padded.shape, A_inputs_padded.shape, A_targets_padded.shape

((11823, 15), (11823, 15), (11823, 15))

### 모델 생성 및 학습

In [ ]:
for i, vects in enumerate(df.values):
    print(vects)   # ['질문', '답변'] 형태로 출력한다.
    if i == 3:
        break

['12시 땡!' '하루가 또 가네요.']
['1지망 학교 떨어졌어' '위로해 드립니다.']
['3박4일 놀러가고 싶다' '여행은 언제나 좋죠.']
['3박4일 정도 놀러가고 싶다' '여행은 언제나 좋죠.']


In [ ]:
import numpy as np
import pandas as pd

def make_embedding_matrix(num_words, embedding_dim, tokenizer, df):

    embedding_matrix = np.zeros((num_words + 1, embedding_dim), dtype=np.float32)

    texts = pd.concat([df['Q'], df['A']]).astype(str).tolist()

    pretrained_embedding = {}
    vocab = tokenizer.word_index.keys()
    for word in vocab:
        pretrained_embedding[word] = np.random.normal(
            loc=0.0, scale=0.05, size=(embedding_dim,)
        ).astype(np.float32)

    # tokenizer 단어 인덱스와 매핑
    for word, index in tokenizer.word_index.items():
        if index > num_words:
            continue
        vects_ = pretrained_embedding.get(word)
        if vects_ is not None:
            embedding_matrix[index] = vects_

    return embedding_matrix


In [18]:
EMBEDDING_DIM = 100

qna_embedding_matrix = make_embedding_matrix(
    q_num_words,
    EMBEDDING_DIM,
    Q_tokenizer,
    df   
)

print(qna_embedding_matrix.shape)


(10001, 100)


In [19]:
q_word_index = ['<pad>'] + list(Q_tokenizer.index_word.values())[:10000]

In [20]:
import pandas as pd

pd.DataFrame(qna_embedding_matrix, index=q_word_index)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
<pad>,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
<OOV>,-0.042541,0.074249,0.117380,0.074609,-0.042105,0.023713,0.018082,0.027844,0.031752,0.043335,...,0.042901,0.039348,0.010680,0.026275,0.050154,0.012012,-0.009921,0.013622,-0.011234,0.033917
너무,0.049486,0.005963,0.021781,0.063960,0.049225,0.010017,-0.009300,-0.011502,-0.065634,0.060040,...,0.082181,0.035144,-0.117430,0.047538,-0.016796,-0.011328,-0.034660,0.062217,0.012783,-0.036920
좋아하는,0.091493,0.022195,-0.001610,-0.020099,-0.088268,-0.040278,0.064456,-0.063074,-0.009800,-0.091817,...,-0.093083,0.080091,-0.010653,-0.017596,0.073729,0.030837,-0.026070,0.054426,0.020693,-0.056653
거,0.086222,0.044359,-0.037667,0.064226,0.038595,0.018800,-0.035402,-0.048974,-0.037487,-0.045912,...,-0.022312,0.068568,-0.050065,-0.006668,-0.058207,0.020351,0.034420,-0.046811,-0.026853,0.017475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
남자들의,-0.141546,0.044652,-0.034394,0.011803,-0.030309,0.132078,-0.089134,0.003070,0.069940,-0.087952,...,-0.070514,0.055980,0.016964,0.016148,0.061721,-0.023099,0.072831,-0.040339,0.023678,-0.035215
이별할때는,-0.016174,-0.071335,0.029518,0.034737,0.027418,-0.003051,-0.012686,0.030409,-0.014167,-0.021306,...,0.040587,0.075810,-0.001078,0.023524,-0.062027,-0.027671,0.048453,0.067351,-0.069462,0.100925
갖지,-0.076492,0.057769,-0.020232,-0.068360,0.033907,0.006152,-0.024582,-0.004494,0.025986,0.011315,...,0.098123,-0.054118,0.018182,0.019961,-0.011474,-0.068503,0.007346,-0.078768,0.010742,0.003598
않게,0.013372,0.122001,0.051015,0.093813,0.032818,0.022463,-0.055821,-0.054341,0.069925,-0.012800,...,0.107498,-0.038478,-0.014225,-0.021886,0.028652,-0.015753,0.003900,-0.042451,0.027515,0.035647


##### 인코더 생성

In [21]:
from tensorflow.keras import layers, models

LATENT_DIM = 512

encoder_inputs = layers.Input(shape=(q_max_len,))
en_embedding_layer = layers.Embedding(q_num_words+1, EMBEDDING_DIM, weights=[qna_embedding_matrix])
x = en_embedding_layer(encoder_inputs)
encoder_outputs, h, c = layers.LSTM(LATENT_DIM, return_state=True)(x)
encoder_states = [h, c]

encoder_model = models.Model(inputs=encoder_inputs, outputs=encoder_states)
encoder_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 15)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 100)        │     1,000,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ [(None, 512), (None,   │     1,255,424 │
│                                 │ 512), (None, 512)]     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,255,524 (8.60 MB)

 Trainable params: 2,255,524 (8.60 MB)

 Non-trainable params: 0 (0.00 B)

##### 디코더(teacher-forcing 모델) 생성

In [22]:
decoder_inputs = layers.Input(shape=[a_max_len,])
ko_embedding_layer = layers.Embedding(a_num_words+1, EMBEDDING_DIM)
x = ko_embedding_layer(decoder_inputs)

decoder_lstm = layers.LSTM(LATENT_DIM, return_sequences=True, return_state=True)
x, h, c = decoder_lstm(x, initial_state=encoder_states)

decoder_dense = layers.Dense(a_num_words+1, activation='softmax')
decoder_outputs = decoder_dense(x)

decoder_teacher_forcing_model = models.Model(
    inputs = [encoder_inputs, decoder_inputs],
    outputs = decoder_outputs
)

decoder_teacher_forcing_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 15, 100)   │  1,000,100 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 15, 100)   │  1,000,100 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 512),     │  1,255,424 │ embedding[0][0]   │
│                     │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 15, 512), │  1,255,424 │ embedding_1[0][0… │
│                     │ (None, 512),      │            │ lstm[0][1],       │
│                     │ (None, 512)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 15, 10001) │  5,130,513 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,641,561 (36.78 MB)

 Trainable params: 9,641,561 (36.78 MB)

 Non-trainable params: 0 (0.00 B)

##### 학습

In [23]:
decoder_teacher_forcing_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

history = decoder_teacher_forcing_model.fit(
    [Q_inputs_padded, A_inputs_padded],
    A_targets_padded,
    batch_size = 64,
    epochs=10,
    validation_split=0.2
)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 71s 464ms/step - accuracy: 0.7390 - loss: 2.4929 - val_accuracy: 0.7134 - val_loss: 2.6142
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 65s 437ms/step - accuracy: 0.7651 - loss: 1.9033 - val_accuracy: 0.7136 - val_loss: 2.5984
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 409ms/step - accuracy: 0.7652 - loss: 1.8130 - val_accuracy: 0.7141 - val_loss: 2.6177
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 61s 411ms/step - accuracy: 0.7665 - loss: 1.7514 - val_accuracy: 0.7119 - val_loss: 2.6647
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 72s 488ms/step - accuracy: 0.7695 - loss: 1.6995 - val_accuracy: 0.7075 - val_loss: 2.7167
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 84s 568ms/step - accuracy: 0.7725 - loss: 1.6485 - val_accuracy: 0.7081 - val_loss: 2.7456
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 64s 435ms/step - accuracy: 0.7744 - loss: 1.5951 - val_accuracy: 0.7077 - val_loss: 2.7864
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 62s 417ms/step - accuracy: 0.7771 - loss: 1

### 모델 추론

### 디코더 (추론 모델) 생성

In [24]:
# from os import initgroups

decoder_hidden_state = layers.Input(shape=(LATENT_DIM,))
decoder_cell_state = layers.Input(shape=(LATENT_DIM,))
decoder_states_inputs = [decoder_hidden_state, decoder_cell_state]

decoder_single_input = layers.Input(shape=(1,))

x = ko_embedding_layer(decoder_single_input)

x, h, c = decoder_lstm(x, initial_state = decoder_states_inputs)
decoder_states = [h, c]

decoder_outputs_ = decoder_dense(x)

decoder_interence_model = models.Model(
    inputs=[decoder_single_input] + decoder_states_inputs,
    outputs=[decoder_outputs_] + decoder_states
)
decoder_interence_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 100)    │  1,000,100 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 1, 512),  │  1,255,424 │ embedding_1[1][0… │
│                     │ (None, 512),      │            │ input_layer_2[0]… │
│                     │ (None, 512)]      │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1, 10001)  │  5,130,513 │ lstm_1[1][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,386,037 (28.18 MB)

 Trainable params: 7,386,037 (28.18 MB)

 Non-trainable params: 0 (0.00 B)

### 추론 함수

In [25]:
def translate(input_seq):        # 번역 함수
  encoder_states_value = encoder_model.predict(input_seq)
  decoder_states_value= encoder_states_value

  bos_index = A_tokenizer.word_index['<bos>']
  eos_index = A_tokenizer.word_index['<eos>']

  target_seq = np.zeros((1, 1))  # 초기화
  target_seq[0, 0] = bos_index

  output_sentences = []

  for _ in range(a_max_len):
    output_tokens, h, c = decoder_interence_model.predict([target_seq] + decoder_states_value)

    pred_proba = output_tokens[0, 0, :]
    pred_index = np.argmax(pred_proba)

    if pred_index == eos_index:
      break

    if pred_index > 0:
      word = A_tokenizer.index_word[pred_index]
      output_sentences.append(word)

    target_seq[0, 0] = pred_index # 현재로 변경
    decoder_states_value = [h, c]
  return " ".join(output_sentences)

### 테스트

In [26]:
for _ in range(5):
  idx = np.random.choice(len(A_inputs_padded))
  input_seq = A_inputs_padded[idx:idx+1]
  output_sent = translate(input_seq)
  display("입력 답:", A_inputs[idx])
  display("학습한 질문:", Q_inputs[idx])
  display("추론한 국문:", output_sent)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


'입력 답:'

'<bos> 나를 사랑하고 상대를 사랑하는 여유를 갖게 되길 바라요.'

'학습한 질문:'

'여유가 너무 없어'

...

'<bos> 많이 사랑한 사람인가봐요.'

'학습한 질문:'

'잘해준것 같은데도 느끼는 미안함'

'추론한 국문:'

'좋은 좋죠.'

### 간단한 Chatbot 구현

1. 사용자의 입력을 받아 (처리)
2. 추론 함수에 전달해서
3. 응답을 출력
4. 1~3 '종료' 전까지 반복

In [27]:
for _ in range(5):
  idx = np.random.choice(len(Q_inputs_padded))
  input_seq = Q_inputs_padded[idx:idx+1]
  output_sent = translate(input_seq)
  display("입력 질문:", Q_inputs[idx])
  display("학습한 답:", A_inputs[idx])
  display("추론한 답:", output_sent)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


'입력 질문:'

'선물로 인형 받았어'

'학습한 답:'

'<bos> 고맙다고 말해 주세요.'

...

'썸 타는 사람이 셀카 보내는 건 뭐야?'

'학습한 답:'

'<bos> 봐달라는 거겠죠.'

'추론한 답:'

'좋은 자신에 대해서 더 공부해서 자신감을 가져보세요.'